# Data processing

#### Requirements

- [TextHero](https://github.com/jbesomi/texthero)
- [langdetect](https://pypi.org/project/langdetect/)
- [datasketch](https://pypi.org/project/datasketch/)
- pandas 
- numpy
- matplotlib
- re
- time



In [1]:
import re
import time
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import texthero as hero
from texthero import stopwords
from texthero import preprocessing

from langdetect import detect
from datasketch import MinHash, MinHashLSHForest

from ast import literal_eval

In [2]:
data = pd.read_csv("data/books.csv")
data.head(3)

,bookId,title,series,author,rating,description,language,isbn,genres,characters,...,firstPublishDate,awards,numRatings,ratingsByStars,likedPercent,setting,coverImg,bbeScore,bbeVotes,price
0,2767052-the-hunger-games,The Hunger Games,The Hunger Games #1,Suzanne Collins,4.33,WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,English,9780439023481,"['Young Adult', 'Fiction', 'Dystopia', 'Fantas...","['Katniss Everdeen', 'Peeta Mellark', 'Cato (H...",...,NaN,['Locus Award Nominee for Best Young Adult Boo...,6376780,"['3444695', '1921313', '745221', '171994', '93...",96.0,"['District 12, Panem', 'Capitol, Panem', 'Pane...",https://i.gr-assets.com/images/S/compressed.ph...,2993816,30516,5.09
1,2.Harry_Potter_and_the_Order_of_the_Phoenix,Harry Potter and the Order of the Phoenix,Harry Potter #5,"J.K. Rowling, Mary GrandPré (Illustrator)",4.50,There is a door at the end of a silent corrido...,English,9780439358071,"['Fantasy', 'Young Adult', 'Fiction', 'Magic',...","['Sirius Black', 'Draco Malfoy', 'Ron Weasley'...",...,06/21/03,['Bram Stoker Award for Works for Young Reader...,2507623,"['1593642', '637516', '222366', '39573', '14526']",98.0,['Hogwarts School of Witchcraft and Wizardry (...,https://i.gr-assets.com/images/S/compressed.ph...,2632233,26923,7.38
2,2657.To_Kill_a_Mockingbird,To Kill a Mockingbird,To Kill a Mockingbird,Harper Lee,4.28,The unforgettable novel of a childhood in a sl...,English,9999999999999,"['Classics', 'Fiction', 'Historical Fiction', ...","['Scout Finch', 'Atticus Finch', 'Jem Finch', ...",...,07/11/60,"['Pulitzer Prize for Fiction (1961)', 'Audie A...",4501075,"['2363896', '1333153', '573280', '149952', '80...",95.0,"['Maycomb, Alabama (United States)']",https://i.gr-assets.com/images/S/compressed.ph...,2269402,23328,NaN


In [3]:
data.shape

(52478, 25)

Initially, there are over 50'000 records.

## Sanitize book formats

In [4]:
counts = data['bookFormat'].nunique()
counts

136

In [5]:
print(data['bookFormat'].unique())

['Hardcover' 'Paperback' 'Mass Market Paperback' 'Kindle Edition'
 'Audiobook' 'ebook' nan 'Board book' 'Boxed Set' 'Leather Bound'
 'Capa dura' 'Trade Paperback' 'Box Set' 'Board Book' 'Nook'
 'Library Binding' 'Capa comum' 'Pasta blanda' 'Audio Cassette'
 'Unknown Binding' 'Audio CD' 'Slipcased Hardcover' 'Broschiert'
 'Paperback ' 'Brochura' 'MP3 CD' 'Audible Audio' 'hardcover' 'cloth'
 'Pasta dura' 'Paperback/Kindle' 'paper' 'Hard Cover' 'Perfect Paperback'
 'Poche' 'Comics' 'Hardcover Slipcased ' 'Unbound' 'Taschenbuch'
 'Paper back' 'Paperback, Kindle, Ebook, Audio' 'CD-ROM'
 'Paperback and Kindle' 'Hardcover im Schuber' 'paperback'
 'Graphic Novels' 'Broché' 'Science Fiction Book Club Omnibus' 'Newsprint'
 'Spiral-bound' 'Mass Market' 'Hardcover Boxed Set'
 'Mass Market Paperback ' 'Hardback' 'Audio' 'Novel' 'Gebundene Ausgabe'
 'softcover' 'گالینگور-وزیری' 'hardbound' 'Hard cover, Soft cover, e-book'
 'Kindle' 'Paperback/Ebook' 'Online Fiction' 'Interactive ebook'
 'Paperback m

In [6]:
# set all names to lower case
data['bookFormat'] = data['bookFormat'].str.lower()
data['bookFormat'] = data['bookFormat'].fillna('unknown binding')

cover_types = data[['bookFormat', 'title']]
cover_types.insert(1, 'count', 1)

n_uniques = cover_types[['bookFormat', 'count']].groupby('bookFormat').sum()
n_uniques.head()

,count
bookFormat,
album,1
audible audio,33
audio,16
audio book,1
audio cassette,36


We can see most of the book formats are either the same or are very marginal. Let's group together into four major groups and put all the other groups into an 'other' category.

In [7]:
n_uniques = n_uniques[n_uniques['count'] > 10] 
n_uniques

,count
bookFormat,
audible audio,33
audio,16
audio cassette,36
audio cd,146
audiobook,107
board book,37
comics,17
ebook,2547
hardcover,12278


In [8]:
format_names = ['hardcover', 'paperback', 'audiobook', 'ebook']
all_cover_names = []
format_categories = [
    ['hardcover', 'slipcased hardcover', 'hardcover slipcased', 'hardcover im schuber', 'hardcover boxed set', 'hardcover, paper dust jacket', 'hardcover chapbook', 'tankobon hardcover'],
    ['mass market paperback', 'paperback', 'trade paperback', 'paperback/ebook', 'paperback, ebook', 'paperback/kindle'],
    ['audible audio', 'audio', 'audio cassette', 'audiobook', 'audio play', 'audio cd'],
    ['ebook', 'kindle edition', 'interactive ebook', 'softcover, free ebook', 'kindle_edition', 'pdf']
]

for i, cover_names in enumerate(format_categories):
    data[format_names[i]] = data.apply(lambda row: row['bookFormat'] in cover_names, axis=1)
    all_cover_names.extend(cover_names)
    
data['other'] = data.apply(lambda row: row['bookFormat'] not in all_cover_names, axis=1)

In [9]:
data[['bookFormat', 'hardcover', 'paperback', 'audiobook', 'ebook', 'other']].sample(10)

,bookFormat,hardcover,paperback,audiobook,ebook,other
33049,hardcover,True,False,False,False,False
23666,paperback,False,True,False,False,False
38796,kindle edition,False,False,False,True,False
13040,paperback,False,True,False,False,False
50199,ebook,False,False,False,True,False
52309,mass market paperback,False,True,False,False,False
4318,paperback,False,True,False,False,False
33767,paperback,False,True,False,False,False
39930,paperback,False,True,False,False,False
35784,hardcover,True,False,False,False,False


## Sanitize non-valid book descriptions

As an important part of the project relies on the similarities between the book's descriptions, we require them to be non-null. Let's filter out the NaN values.

In [10]:
data.shape

(52478, 30)

In [11]:
MIN_NB_CHAR = 40

data = data[data['description'].notna()]

# checks if the description contains at least one latin alphabetical character AND if the description is at least 40 characters long
data['is_descr_valid'] = data.apply(lambda row: len(row['description'])>MIN_NB_CHAR and bool(re.match('^(?=.*[a-zA-Z])', row['description'])), axis=1)

data = data[data['is_descr_valid']==True]
data = data.drop(['is_descr_valid'], axis = 1)

In [12]:
data.shape

(48940, 30)

This only removes about 6% of the dataset which is acceptable.

# Sanitize book language

In [13]:
data_not_en = data[data['language'] != 'English']
data_not_en.shape

(7521, 30)

Around 15% of the dataset does not have English langage set, this is non-negligeable.

In [14]:
data_not_en.sample(5)

,bookId,title,series,author,rating,description,language,isbn,genres,characters,...,setting,coverImg,bbeScore,bbeVotes,price,hardcover,paperback,audiobook,ebook,other
18619,1835592.Flesh_and_Blood,Flesh and Blood,NaN,Graham Masterton,3.77,"America's biggest pig, Captain Black, undergoe...",NaN,9780434000142,"['Horror', 'Fiction']",[],...,[],https://i.gr-assets.com/images/S/compressed.ph...,100,1,3.49,True,False,False,False,False
38105,27840243-minu-kennedy,Minu Kennedy,NaN,Stig Rästa,3.83,"""Kirjutan sulle esimest korda. Ma ei tea su va...",Estonian,9789949387410,"['Biography', 'Nonfiction']",[],...,[],https://i.gr-assets.com/images/S/compressed.ph...,86,1,NaN,False,True,False,False,False
20499,29546259-raden-mandasia-si-pencuri-daging-sapi,Raden Mandasia si Pencuri Daging Sapi,NaN,Yusi Avianto Pareanom,4.57,SUNGU LEMBU menjalani hidup membawa dendam. Ra...,Indonesian,9789791079525,"['Fiction', 'Fantasy', 'Indonesian Literature'...",[],...,[],https://i.gr-assets.com/images/S/compressed.ph...,100,1,NaN,False,True,False,False,False
23180,39293656-tears-of-glass,Tears of Glass,NaN,David Lake (Goodreads Author),3.93,"A failed quarterback, failed husband and faile...",NaN,B01CHTF3HQ,"['Fiction', 'Thriller']",[],...,[],https://i.gr-assets.com/images/S/compressed.ph...,99,1,NaN,False,False,False,True,False
19075,18388163-nightmarish-sacrifice,Nightmarish Sacrifice,NaN,Simona Panova (Goodreads Author),4.33,Unpredictable. Symbolic. Mysterious. NIGHTMARI...,NaN,9781491021408,[],[],...,[],https://i.gr-assets.com/images/S/compressed.ph...,100,1,NaN,False,True,False,False,False


#### We can see a lot of books with NaN values still have their descriptions in english, let's quickly use Google's language-detection package on the descriptions to infer if the books are indeed in English

In [15]:
data['is_description_en'] = data.apply(lambda row: row['description']=='English' or detect(row['description'])=='en', axis=1)

In [16]:
# Let's drop the books whose summary has not been detected to be in English
data = data[data['is_description_en']==True]

In [17]:
data.shape

(44800, 31)

## Sanitize books without any genres 

Our main visualization will greatly depend on the genres of each displayed book. We will therefore require the valid books to have at least one genre. Furthermore, in order to not duplicate the books displays in our bubble graph, we will stick with the first main genre of each book to represent it.

In [18]:
data.shape

(44800, 31)

In [19]:
# Remove books with NaN values on 'genres'
data = data[data['genres'].notna()]

In [20]:
# Remove books with less than one genre
data['nb_genres'] = data.apply(lambda row: len(literal_eval(row['genres'])), axis=1)
data = data[data['nb_genres']>=1]
data = data.drop(['nb_genres'], axis = 1)

In [21]:
# Derive main genre of each book into separate column
data['first_genre'] = data.apply(lambda row: literal_eval(row['genres'])[0], axis=1)

In [22]:
data.shape

(41478, 32)

This procedure removed 3320 books (about 6% of the dataset), which is acceptable.

## Infer missing prices and number of pages

In the website, we filter the dataset based on some widgets levels. For this reason we want every book from the dataset to have a value for the concerned fields. Setting the NaN to the median has low impact and allows to keep most books potentially displayable if the user sets average values on the widgets.

#### a) Prices

In [23]:
# Replace a few corrupted datapoints by NaN values
data['price'] = data['price'].replace(['1.743.28'],np.NaN)
data['price'] = data['price'].replace(['1.307.46'],np.NaN)
data['price'] = data['price'].replace(['8.715.51'],np.NaN)
data['price'] = data['price'].replace(['1.775.18'],np.NaN)
data['price'] = data['price'].replace(['1.734.84'],np.NaN)

In [24]:
no_nan_price = data[data['price'].notna()]
no_nan_price['price'] = pd.to_numeric(no_nan_price['price'])

C:\Users\GuillaumeP\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [25]:
max_price = no_nan_price['price'].max()
max_price

898.64

In [26]:
min_price = no_nan_price['price'].min()
min_price

0.84

In [27]:
median_price = no_nan_price['price'].median()
median_price

5.09

In [28]:
data['price'] = data['price'].fillna(median_price)

#### b) Number of pages

In [29]:
# Replace a few corrupted datapoints
data['pages'] = data['pages'].replace(['1 page'],1)

In [30]:
with_nb_pages = data[data['pages'].notna()]
with_nb_pages['pages'] = pd.to_numeric(with_nb_pages['pages'])

C:\Users\GuillaumeP\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [31]:
max_price = with_nb_pages['pages'].max()
max_price

14777

In [32]:
min_price = with_nb_pages['pages'].min()
min_price

0

In [33]:
median_price = with_nb_pages['pages'].median()
median_price

312.0

In [34]:
data['pages'] = data['pages'].fillna(median_price)

## Export usefull pre-processed data

In [35]:
# Fields currently used on the website, can be adapted to future needs
usefull_fields = [
    'bookId',
    'title',
    'author',
    'first_genre',
    'coverImg',
    'rating',
    'numRatings',
    'price',
    'description',
    'pages',
    'publishDate',
    'publisher',
    'bookFormat',
    'hardcover',
    'paperback',
    'audiobook',
    'ebook',
    'other',
    'isbn'
]

export_data = data[usefull_fields]

In [36]:
export_data.head(5)

,bookId,title,author,first_genre,coverImg,rating,numRatings,price,description,pages,publishDate,publisher,bookFormat,hardcover,paperback,audiobook,ebook,other,isbn
0,2767052-the-hunger-games,The Hunger Games,Suzanne Collins,Young Adult,https://i.gr-assets.com/images/S/compressed.ph...,4.33,6376780,5.09,WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,374,09/14/08,Scholastic Press,hardcover,True,False,False,False,False,9780439023481
1,2.Harry_Potter_and_the_Order_of_the_Phoenix,Harry Potter and the Order of the Phoenix,"J.K. Rowling, Mary GrandPré (Illustrator)",Fantasy,https://i.gr-assets.com/images/S/compressed.ph...,4.50,2507623,7.38,There is a door at the end of a silent corrido...,870,09/28/04,Scholastic Inc.,paperback,False,True,False,False,False,9780439358071
2,2657.To_Kill_a_Mockingbird,To Kill a Mockingbird,Harper Lee,Classics,https://i.gr-assets.com/images/S/compressed.ph...,4.28,4501075,5.09,The unforgettable novel of a childhood in a sl...,324,05/23/06,Harper Perennial Modern Classics,paperback,False,True,False,False,False,9999999999999
3,1885.Pride_and_Prejudice,Pride and Prejudice,"Jane Austen, Anna Quindlen (Introduction)",Classics,https://i.gr-assets.com/images/S/compressed.ph...,4.26,2998241,5.09,Alternate cover edition of ISBN 9780679783268S...,279,10/10/00,Modern Library,paperback,False,True,False,False,False,9999999999999
4,41865.Twilight,Twilight,Stephenie Meyer,Young Adult,https://i.gr-assets.com/images/S/compressed.ph...,3.60,4964519,2.1,About three things I was absolutely positive.\...,501,09/06/06,"Little, Brown and Company",paperback,False,True,False,False,False,9780316015844


In [37]:
export_data.to_csv(path_or_buf='./data/processed_books.csv', index=False)
# export_data.to_pickle(path='./data/processed_books.pkl')

## Export Tinder books data

In [38]:
tinder_30book_ids = [
    '2767052-the-hunger-games',
    '2.Harry_Potter_and_the_Order_of_the_Phoenix',
    '2657.To_Kill_a_Mockingbird',
    '1885.Pride_and_Prejudice',
    '33.The_Lord_of_the_Rings',
    '370493.The_Giving_Tree',
    '968.The_Da_Vinci_Code',
    '24213.Alice_s_Adventures_in_Wonderland_Through_the_Looking_Glass',
    '24280.Les_Mis_rables',
    '18144590-the-alchemist',
    '7144.Crime_and_Punishment',
    '22628.The_Perks_of_Being_a_Wallflower',
    '375802.Ender_s_Game',
    '17245.Dracula',
    '13496.A_Game_of_Thrones',
    '1381.The_Odyssey',
    '4214.Life_of_Pi',
    '44767458-dune',
    '3590.The_Adventures_of_Sherlock_Holmes',
    '2429135.The_Girl_with_the_Dragon_Tattoo',
    '4934.The_Brothers_Karamazov',
    '99107.Winnie_the_Pooh',
    '49552.The_Stranger',
    '11588.The_Shining',
    '99561.Looking_for_Alaska',
    '1618.The_Curious_Incident_of_the_Dog_in_the_Night_Time',
    '119073.The_Name_of_the_Rose',
    '22034.The_Godfather',
    '830502.It',
    '1845.Into_the_Wild', 
]

tinder_books_df = data[data['bookId'].isin(tinder_30book_ids)]
tinder_books_df.shape

(30, 32)

In [39]:
tinder_books_df.head(30)

,bookId,title,series,author,rating,description,language,isbn,genres,characters,...,bbeScore,bbeVotes,price,hardcover,paperback,audiobook,ebook,other,is_description_en,first_genre
0,2767052-the-hunger-games,The Hunger Games,The Hunger Games #1,Suzanne Collins,4.33,WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,English,9780439023481,"['Young Adult', 'Fiction', 'Dystopia', 'Fantas...","['Katniss Everdeen', 'Peeta Mellark', 'Cato (H...",...,2993816,30516,5.09,True,False,False,False,False,True,Young Adult
1,2.Harry_Potter_and_the_Order_of_the_Phoenix,Harry Potter and the Order of the Phoenix,Harry Potter #5,"J.K. Rowling, Mary GrandPré (Illustrator)",4.50,There is a door at the end of a silent corrido...,English,9780439358071,"['Fantasy', 'Young Adult', 'Fiction', 'Magic',...","['Sirius Black', 'Draco Malfoy', 'Ron Weasley'...",...,2632233,26923,7.38,False,True,False,False,False,True,Fantasy
2,2657.To_Kill_a_Mockingbird,To Kill a Mockingbird,To Kill a Mockingbird,Harper Lee,4.28,The unforgettable novel of a childhood in a sl...,English,9999999999999,"['Classics', 'Fiction', 'Historical Fiction', ...","['Scout Finch', 'Atticus Finch', 'Jem Finch', ...",...,2269402,23328,5.09,False,True,False,False,False,True,Classics
3,1885.Pride_and_Prejudice,Pride and Prejudice,NaN,"Jane Austen, Anna Quindlen (Introduction)",4.26,Alternate cover edition of ISBN 9780679783268S...,English,9999999999999,"['Classics', 'Fiction', 'Romance', 'Historical...","['Mr. Bennet', 'Mrs. Bennet', 'Jane Bennet', '...",...,1983116,20452,5.09,False,True,False,False,False,True,Classics
12,370493.The_Giving_Tree,The Giving Tree,NaN,Shel Silverstein,4.37,"""Once there was a tree...and she loved a littl...",English,9780060256654,"['Childrens', 'Picture Books', 'Classics', 'Fi...",[],...,1021534,10594,4.87,True,False,False,False,False,True,Childrens
14,968.The_Da_Vinci_Code,The Da Vinci Code,Robert Langdon #2,Dan Brown (Goodreads Author),3.86,ISBN 9780307277671 moved to this edition.While...,English,9999999999999,"['Fiction', 'Mystery', 'Thriller', 'Suspense',...","['Sophie Neveu', 'Robert Langdon', 'Sir Leigh ...",...,876633,9231,5.09,False,True,False,False,False,True,Fiction
17,24213.Alice_s_Adventures_in_Wonderland_Through...,Alice's Adventures in Wonderland & Through the...,Alice's Adventures in Wonderland #1-2,"Lewis Carroll, John Tenniel (Illustrator), Mar...",4.06,"""I can't explain myself, I'm afraid, sir,"" sai...",English,9780451527745,"['Classics', 'Fantasy', 'Fiction', 'Childrens'...","['The Hatter (Lewis Carroll)', 'The Queen of H...",...,833791,8812,3.07,False,True,False,False,False,True,Classics
19,24280.Les_Mis_rables,Les Misérables,NaN,"Victor Hugo, Lee Fahnestock (Translator), Norm...",4.18,Introducing one of the most famous characters ...,English,9999999999999,"['Classics', 'Fiction', 'Historical Fiction', ...","['Jean Valjean', 'Javert', 'Cosette', 'Fantine...",...,813088,8548,5.09,False,True,False,False,False,True,Classics
24,18144590-the-alchemist,The Alchemist,NaN,"Paulo Coelho (Goodreads Author), Alan R. Clark...",3.88,Paulo Coelho's enchanting novel has inspired a...,English,9780062315007,"['Fiction', 'Classics', 'Fantasy', 'Philosophy...","['Santiago', 'Alchemist', 'Melchizedek']",...,765587,8008,13.22,False,True,False,False,False,True,Fiction
25,7144.Crime_and_Punishment,Crime and Punishment,NaN,"Fyodor Dostoyevsky, David McDuff (Translator)",4.22,"Raskolnikov, a destitute and desperate former ...",English,9780143058144,"['Classics', 'Fiction', 'Russia', 'Literature'...","['Rodion Romanovich Raskolnikov', 'Porfiry Pet...",...,759066,7937,18.85,False,True,False,False,False,True,Classics


In [40]:
tinder_books_df[['bookId', 'coverImg']].to_csv(path_or_buf='./data/tinder_books.csv', index=False)

In [1]:
import re
import time
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import texthero as hero
from texthero import stopwords
from texthero import preprocessing

from langdetect import detect
from datasketch import MinHash, MinHashLSHForest

from ast import literal_eval

In [2]:
data = pd.read_pickle('../processed_books.pkl')

In [3]:
data.head()

,bookId,title,author,first_genre,coverImg,rating,numRatings,price,description,pages,publishDate,publisher,bookFormat,hardcover,paperback,audiobook,ebook,other,isbn
0,2767052-the-hunger-games,The Hunger Games,Suzanne Collins,Young Adult,https://i.gr-assets.com/images/S/compressed.ph...,4.33,6376780,5.09,WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,374,09/14/08,Scholastic Press,hardcover,True,False,False,False,False,9780439023481
1,2.Harry_Potter_and_the_Order_of_the_Phoenix,Harry Potter and the Order of the Phoenix,"J.K. Rowling, Mary GrandPré (Illustrator)",Fantasy,https://i.gr-assets.com/images/S/compressed.ph...,4.50,2507623,7.38,There is a door at the end of a silent corrido...,870,09/28/04,Scholastic Inc.,paperback,False,True,False,False,False,9780439358071
2,2657.To_Kill_a_Mockingbird,To Kill a Mockingbird,Harper Lee,Classics,https://i.gr-assets.com/images/S/compressed.ph...,4.28,4501075,5.09,The unforgettable novel of a childhood in a sl...,324,05/23/06,Harper Perennial Modern Classics,paperback,False,True,False,False,False,9999999999999
3,1885.Pride_and_Prejudice,Pride and Prejudice,"Jane Austen, Anna Quindlen (Introduction)",Classics,https://i.gr-assets.com/images/S/compressed.ph...,4.26,2998241,5.09,Alternate cover edition of ISBN 9780679783268S...,279,10/10/00,Modern Library,paperback,False,True,False,False,False,9999999999999
4,41865.Twilight,Twilight,Stephenie Meyer,Young Adult,https://i.gr-assets.com/images/S/compressed.ph...,3.60,4964519,2.1,About three things I was absolutely positive.\...,501,09/06/06,"Little, Brown and Company",paperback,False,True,False,False,False,9780316015844


## Compute description similarities

In [46]:
help(hero)

Help on package texthero:

NAME
    texthero - Texthero: python toolkit for text preprocessing, representation and visualization.

PACKAGE CONTENTS
    nlp
    preprocessing
    representation
    stopwords
    visualization

DATA
    Callable = typing.Callable
    List = typing.List
    Optional = typing.Optional
    Set = typing.Set

FILE
    c:\users\guillaumep\anaconda3\lib\site-packages\texthero\__init__.py




In [62]:
# Perform standard cleaning operations on the books summary
custom_pipeline = [preprocessing.fillna,
                   preprocessing.lowercase,
                   preprocessing.remove_digits,
                   preprocessing.remove_punctuation,
                   preprocessing.remove_urls,
                   preprocessing.remove_html_tags,
                   preprocessing.remove_diacritics, # All special accents like Noël
                   preprocessing.remove_whitespace] # Remove any extra whitespace, newline, tabs and any form of space.
data['nlp_description'] = hero.clean(data['description'], custom_pipeline)

# Add a few custom stopwords to be removed
default_stopwords = stopwords.DEFAULT
specific_stopwords = ['isbn', 'my', 'great', 'interesting', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
custom_stopwords = default_stopwords.union(set(specific_stopwords))

# Reapply some operations after removal of stopwords
data['nlp_description'] = hero.remove_stopwords(data.nlp_description, custom_stopwords)
data['nlp_description'] = hero.remove_punctuation(data.nlp_description)
data['nlp_description'] = hero.remove_digits(data.nlp_description)

# Apply some stemming
data['nlp_description'] = hero.stem(data.nlp_description)

In [63]:
data[['description', 'nlp_description']].head(10)

,description,nlp_description
0,WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,win mean fame fortun lose mean certain death h...
1,There is a door at the end of a silent corrido...,door end silent corridor haunt harri pottter d...
2,The unforgettable novel of a childhood in a sl...,unforgett novel childhood sleepi southern town...
3,Alternate cover edition of ISBN 9780679783268S...,altern cover edit 9780679783268sinc immedi suc...
4,About three things I was absolutely positive.\...,three thing absolut posit first edward vampir ...
5,Librarian's note: An alternate cover edition c...,librarian note altern cover edit found hereit ...
6,Librarian's note: There is an Alternate Cover ...,librarian note altern cover edit edit book far...
7,"Journeys to the end of the world, fantastic cr...",journey end world fantast creatur epic battl g...
8,"This four-volume, boxed set contains J.R.R. To...",four volum box set contain tolkien epic master...
9,"Scarlett O'Hara, the beautiful, spoiled daught...",scarlett hara beauti spoil daughter well georg...


In [54]:
descriptions_tfidf = hero.tfidf(data['nlp_description'], min_df=0.0001)

In [57]:
len(descriptions_tfidf[0])

20043

In [60]:
descriptions_tfidf_tuned = hero.tfidf(data['nlp_description'], max_features=10000) #, min_df=4.7e-5)

MemoryError: Unable to allocate 12.4 GiB for an array with shape (41478, 40000) and data type float64

In [78]:
descriptions_tfidf_tuned = hero.tfidf(data['nlp_description'], min_df=50)

MemoryError: 

In [ ]:
len(descriptions_tfidf_tuned[0])

In [ ]:
# Have at most 75880 words in vocabulary

In [43]:
tinder_books_df[['description', 'nlp_description']].head(10)

,description,nlp_description
0,WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,win mean fame fortun lose mean certain death h...
1,There is a door at the end of a silent corrido...,door end silent corridor haunt harri pottter d...
2,The unforgettable novel of a childhood in a sl...,unforgett novel childhood sleepi southern town...
3,Alternate cover edition of ISBN 9780679783268S...,altern cover edit isbn 9780679783268sinc immed...
12,"""Once there was a tree...and she loved a littl...",tree love littl boy begin stori unforgett perc...
14,ISBN 9780307277671 moved to this edition.While...,isbn move edit pari harvard symbologist robert...
17,"""I can't explain myself, I'm afraid, sir,"" sai...",explain afraid sir said alic see alic see whit...
19,Introducing one of the most famous characters ...,introduc one famous charact literatur jean val...
24,Paulo Coelho's enchanting novel has inspired a...,paulo coelho enchant novel inspir devot follow...
25,"Raskolnikov, a destitute and desperate former ...",raskolnikov destitut desper former student wan...


In [44]:
descriptions_tfidf_tinder = hero.tfidf(tinder_books_df['nlp_description'], return_feature_names=False)

In [45]:
descriptions_tfidf_tinder

0      [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
1      [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
2      [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1112975272838...
3      [0.13165710973501074, 0.0, 0.0, 0.0, 0.0, 0.0,...
12     [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.10842562...
14     [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
17     [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
19     [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
24     [0.0, 0.0, 0.0, 0.0, 0.11120532695454781, 0.0,...
25     [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
26     [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
29     [0.0, 0.0, 0.0, 0.06771898023724653, 0.0, 0.0,...
37     [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
46     [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
52     [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1349236240616...
53     [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
63     [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
66     [0.0, 0.0, 0.0, 0.0, 0.0

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
documents = [open(f).read() for f in text_files]
tfidf = TfidfVectorizer().fit_transform(documents)
# no need to normalize, since Vectorizer will return normalized tf-idf
pairwise_similarity = tfidf * tfidf.T

In [ ]:
x.toarray()

In [ ]:
def jaccard_similarity(x,y):
    """ returns the jaccard similarity between two lists """
    intersection_cardinality = len(set.intersection(*[set(x), set(y)]))
    union_cardinality = len(set.union(*[set(x), set(y)]))
    return intersection_cardinality/float(union_cardinality)

In [ ]:
"There is nothing in the bottle"]
sentences = [sent.lower().split(" ") for sent in sentences]
jaccard_similarity(sentences[0], sentences[1])

In [ ]:
def get_forest(data, perms):
    start_time = time.time()
    
    minhash = []
    
    for description in data['nlp_description']:
        tokens = description.split()
        m = MinHash(num_perm=perms)
        for s in tokens:
            m.update(s.encode('utf8'))
        minhash.append(m)
        
    forest = MinHashLSHForest(num_perm=perms)
    
    for i,m in enumerate(minhash):
        forest.add(i,m)
        
    forest.index()
    
    print('It took %s seconds to build forest.' %(time.time()-start_time))
    
    return forest

In [ ]:
def predict(description, database, perms, num_results, forest):
    start_time = time.time()
    
    tokens = description.split()
    m = MinHash(num_perm=perms)
    for s in tokens:
        m.update(s.encode('utf8'))
        
    idx_array = np.array(forest.query(m, num_results))
    if len(idx_array) == 0:
        return None
    
    result = database.iloc[idx_array]['title']
    
    print('It took %s seconds to query forest.' %(time.time()-start_time))
    
    return result

In [ ]:
data

In [ ]:
#Number of Permutations
permutations = 128

#Number of Recommendations to return
num_recommendations = 1

forest = get_forest(data, permutations)

In [ ]:
db = pd.read_csv('papers.csv')
db['text'] = db['title'] + ' ' + db['abstract']
forest = get_forest(db, permutations)

In [ ]:
num_recommendations = 5
title = 'Using a neural net to instantiate a deformable model'
result = predict(title, db, permutations, num_recommendations, forest)
print('\n Top Recommendation(s) is(are) \n', result)